### 04. Análisis y auditoría de valores faltantes

Objetivos:
- Cuantificar valores faltantes reales (dataset original).
- Distinguir entre:
  - Faltante genuino (dato que debería existir y no está).
  - Valor *no aplicable* introducido por ingeniería (ej. `Evaluation_cp` es NaN porque hubo mate).
- Verificar si existen patrones (correlación entre máscaras de faltantes).
- Concluir sobre el mecanismo (MCAR / MAR / MNAR) y documentar si se requiere imputación.

Contexto específico del dataset de ajedrez:
- La columna original `Evaluation` se parseó a `Evaluation_cp`, `Evaluation_mate_sign`, `Evaluation_mate_in`.
- Cuando hay mate, `Evaluation_cp` es NaN por diseño (no es realmente un faltante).
- Cuando NO hay mate, `Evaluation_mate_in` es NaN (también "no aplica").
- Por lo tanto la gran mayoría (o totalidad) de los NaN provienen de reglas de transformación.

Criterios de clasificación en este notebook:
- `real_missing`: NaN en columnas originales (si existieran).
- `engineered_na`: NaN proveniente de columnas derivadas con sufijos `_cp`, `_mate_sign`, `_mate_in`.
- `none`: sin valores NaN.
- `mixed_or_derived`: otros casos residuales (si aparecieran nuevas transformaciones).

.> Si no hay faltantes genuinos, se documenta claramente para la trazabilidad del pipeline.

In [ ]:
# Carga directa (versión copiada)
from pathlib import Path
import pandas as pd

DATA_DIR = Path('../data/raw')
CHESS_PATH = DATA_DIR / 'chessData.csv'

def load_chess(nrows=None, usecols=None):
    """Carga chessData.csv con backend pyarrow si disponible.
    - nrows: permite carga parcial para pruebas rápidas.
    - usecols: para limitar columnas.
    """
    read_kw = dict(low_memory=False, on_bad_lines='warn')
    try:
        # pandas >= 2 admite dtype_backend='pyarrow'
        return pd.read_csv(CHESS_PATH, dtype_backend='pyarrow', nrows=nrows, usecols=usecols, **read_kw)
    except TypeError:
        return pd.read_csv(CHESS_PATH, nrows=nrows, usecols=usecols, **read_kw)

df = load_chess()
print(f'Filas: {len(df):,}  | Columnas: {len(df.columns)}')
print('Columnas del df:', list(df.columns[:15]))

# Definir columnas originales si no existen todavía
if 'ORIG_COLUMNS' not in globals():
    ORIG_COLUMNS = [c for c in df.columns if not c.endswith(('_cp','_mate_sign','_mate_in'))]

[INFO] No se pudo importar functions.functions directamente -> ajustando sys.path


[INFO] No se pudo importar functions.functions directamente -> ajustando sys.path


KeyboardInterrupt: 

In [1]:
# --- 04. Análisis y auditoría de valores faltantes --------------------------------
#
# Objetivos:
#  - Cuantificar valores faltantes reales.
#  - Separar NaN introducidos por ingeniería (mate_in, mate_sign, *_cp cuando hay mate).
#  - Evaluar patrones (si los hubiera) y comentar MCAR / MAR / MNAR.
#  - Concluir si se requiere imputación (en este dataset no).
#
# Notas:
#  * NaN en Evaluation_cp cuando hay mate -> NO es "dato perdido"; significa "no aplica (mate)".
#  * Mate_in NaN cuando no hay mate -> también "no aplica".
#  * Solo consideraremos "faltante real" si la columna original tenía NaN antes de generar columnas derivadas.
#
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Guardar columnas originales justo después de cargar (si no se guardó antes)
if 'ORIG_COLUMNS' not in globals():
    ORIG_COLUMNS = [c for c in df.columns if not c.endswith(('_cp','_mate_sign','_mate_in'))]

def analyze_missing_values(
    df: pd.DataFrame,
    original_cols=None,
    engineered_suffixes=('_cp','_mate_sign','_mate_in'),
    show_plot=True
):
    """
    Analiza valores faltantes distinguiendo:
      - missing_real: NaN en columnas originales
      - engineered_na: NaN en columnas derivadas por diseño (no aplica)
    """
    original_cols = original_cols or ORIG_COLUMNS
    
    n = len(df)
    base = pd.DataFrame(index=df.columns)
    base['total'] = n
    base['missing_count'] = df.isna().sum()
    base['missing_pct'] = base['missing_count'] / n * 100
    base['dtype'] = df.dtypes.astype(str)
    
    # Clasificación simple
    def classify(col, miss_cnt):
        if miss_cnt == 0:
            return 'none'
        if col in original_cols:
            return 'real_missing'
        if col.endswith(engineered_suffixes):
            return 'engineered_na'
        return 'mixed_or_derived'
    
    base['missing_type'] = [
        classify(c, base.loc[c,'missing_count']) for c in base.index
    ]
    
    # Separar verdaderos faltantes (solo columnas originales con NaN)
    real_missing = base[(base.missing_type=='real_missing') & (base.missing_count>0)]
    
    if show_plot and len(real_missing) > 0:
        fig, axes = plt.subplots(1, 2, figsize=(14,5))
        rm_sorted = real_missing.sort_values('missing_pct', ascending=False)
        axes[0].bar(rm_sorted.index, rm_sorted['missing_pct'], color='coral')
        axes[0].set_ylabel('% faltante')
        axes[0].set_title('Porcentaje de valores faltantes (reales)')
        axes[0].tick_params(axis='x', rotation=90)
        axes[0].axhline(5, ls='--', color='red', label='Umbral 5%')
        axes[0].legend()
        
        # Mapa de correlación de patrones (solo columnas con faltantes reales)
        miss_pattern = df[rm_sorted.index].isna().astype(int)
        if miss_pattern.shape[1] >= 2:
            sns.heatmap(miss_pattern.corr(), annot=True, fmt='.2f',
                        cmap='coolwarm', vmin=-1, vmax=1, ax=axes[1])
            axes[1].set_title('Correlación patrones de faltantes')
        else:
            axes[1].axis('off')
            axes[1].text(0.5,0.5,'Sólo 1 columna con faltantes reales', ha='center')
        plt.tight_layout()
    elif show_plot:
        print("✅ No hay valores faltantes reales en columnas originales.")
    
    return base.sort_values('missing_pct', ascending=False)

missing_report = analyze_missing_values(df)

display(
    missing_report[['missing_count','missing_pct','missing_type','dtype']]
      .head(25)
      .style.format({'missing_pct':'{:.2f}'})
)

# Resumen textual
total_cols = len(missing_report)
real_cols_with_missing = (missing_report.missing_type=='real_missing') & (missing_report.missing_count>0)
n_real_missing_cols = real_cols_with_missing.sum()
max_pct = missing_report.loc[real_cols_with_missing,'missing_pct'].max() if n_real_missing_cols else 0

print("\n--- Conclusión de auditoría ---")
if n_real_missing_cols == 0:
    print("No se detectaron valores faltantes genuinos en las columnas originales.")
    print("Los NaN presentes pertenecen a columnas derivadas (mate / no aplica).")
    print("No se requiere imputación. Clasificación: ausencia de patrones MCAR/MAR/MNAR porque no hay faltantes reales.")
else:
    print(f"{n_real_missing_cols} columnas originales con faltantes. Máximo porcentaje: {max_pct:.2f}%.")
    print("Inspeccionar si son MCAR (aleatorios), MAR (dependen de otras vars) o MNAR (dependen de su propio valor).")
    print("Aplicar estrategia de imputación adecuada (media/mediana/modelo) sólo si son relevantes para el modelado.")

NameError: name 'df' is not defined

#### Conclusión
- No se identificaron valores faltantes genuinos en las columnas originales del dataset de posiciones de ajedrez.
- Los NaN observados pertenecen a columnas derivadas (`*_cp`, `*_mate_in`) y representan un estado *no aplicable* (mate presente o ausente).
- No procede imputación (evita introducir sesgo artificial).
- Mecanismo: ausencia de faltantes reales ⇒ no aplica clasificación MCAR/MAR/MNAR.

**Siguiente paso:** continuar con ingeniería de variables / creación de etiqueta de ventaja para el modelo de clasificación.

In [ ]:
# Visualización opcional: barras comparando tipos de NA
def plot_missing_type_counts(report):
    counts = report['missing_type'].value_counts()
    if counts.empty:
        print('Sin categorías de faltantes para graficar.')
        return
    ax = counts.plot(kind='bar', color=['#4c72b0','#dd8452','#55a868','#c44e52'])
    ax.set_ylabel('Número de columnas')
    ax.set_title('Conteo de columnas por tipo de faltante')
    for p in ax.patches:
        ax.annotate(int(p.get_height()), (p.get_x()+p.get_width()/2, p.get_height()),
                    ha='center', va='bottom')
    plt.show()

plot_missing_type_counts(missing_report)

In [2]:
# Carga ligera del dataset si 'df' no está en memoria (permite ejecutar este notebook de forma independiente)
from pathlib import Path
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

if 'df' not in globals():
    DATA_DIR = Path('../data/raw')
    CHESS_PATH = DATA_DIR / 'chessData.csv'
    df = pd.read_csv(CHESS_PATH, low_memory=False)
    print('Dataset cargado localmente en este notebook:', df.shape)
else:
    print('Usando DataFrame existente en memoria:', df.shape)

# Mostrar primeras columnas para referencia rápida
print('Columnas (primeras 15):', list(df.columns[:15]))
df.head(3)

Dataset cargado localmente en este notebook: (12958035, 2)
Columnas (primeras 15): ['FEN', 'Evaluation']


,FEN,Evaluation
0,rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR ...,-10
1,rnbqkbnr/pppp1ppp/4p3/8/4P3/8/PPPP1PPP/RNBQKBN...,+56
2,rnbqkbnr/pppp1ppp/4p3/8/3PP3/8/PPP2PPP/RNBQKBN...,-9
